In this notebook we test the difference between dynamic using multitask learning and static using the results from the multitask learning.

In [1]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import Siren, NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold
from multitaskpinn.training import train, train_multitask
from multitaskpinn.training.sparsity_scheduler import TrainTestPeriodic, s_periodic, Periodic

from phimal_utilities.data import Dataset
from phimal_utilities.data.burgers import BurgersDelta

#if torch.cuda.is_available():
#    device ='cuda'
#else:
#    device = 'cpu'
    
device = 'cpu'

# Settings for reproducibility
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

%load_ext autoreload
%autoreload 2

In [2]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)

X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.1, random=True, normalize=True)
X, y = X.to(device), y.to(device)

# Running dynamic

In [3]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

In [4]:
sparsity_scheduler = Periodic(initial_epoch=20000)#s_periodic(patience=100, delta=0.05, periodicity=100)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), amsgrad=True) # Defining optimizer
    
train_multitask(model, X, y, optimizer, sparsity_scheduler, write_iterations=25, log_dir='runs/multitask_dynamic/', max_iterations=10000, delta=0.00, patience=100) # Running

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
      10000    100.00%               0s   -1.70e+01   3.47e-04   1.04e-07   3.06e+00 

In [4]:
model.s

AttributeError: 'DeepMoD' object has no attribute 's'

In [6]:
torch.exp(-model.s)[0, 1] / torch.exp(-model.s)[0, 0]

tensor(7.9959, grad_fn=<DivBackward0>)

So 300 times as big...

# Running static

In [17]:
import torch
import time
from math import pi
import numpy as np

from multitaskpinn.utils.tensorboard import Tensorboard
from multitaskpinn.utils.output import progress
from multitaskpinn.training.convergence import Convergence
from multitaskpinn.model.deepmod import DeepMoD
from typing import Optional

In [18]:
def train_static(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          loss_weight,
          optimizer,
          sparsity_scheduler,
          log_dir: Optional[str] = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """[summary]

    Args:
        model (DeepMoD): [description]
        data (torch.Tensor): [description]
        target (torch.Tensor): [description]
        optimizer ([type]): [description]
        sparsity_scheduler ([type]): [description]
        log_dir (Optional[str], optional): [description]. Defaults to None.
        max_iterations (int, optional): [description]. Defaults to 10000.
    """
    start_time = time.time()
    board = Tensorboard(log_dir)  # initializing tb board

    # Training
    convergence = Convergence(**convergence_kwargs)
    print('| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |')
    for iteration in np.arange(0, max_iterations + 1):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data)

        MSE = torch.mean((prediction - target)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])
        loss = torch.sum(MSE + loss_weight * Reg)  # 1e-5 for numerical stability

        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ====================== Logging =======================
        # We calculate the normalization factor and the l1_norm
        l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)), dim=0)

        # Write progress to command line and tensorboard
        if iteration % write_iterations == 0:
            _ = model.sparse_estimator(thetas, time_derivs) # calculating l1 adjusted coeffs but not setting mask
            progress(iteration, start_time, max_iterations, loss.item(),
                     torch.sum(MSE).item(), torch.sum(Reg).item(), torch.sum(l1_norm).item())

            if model.estimator_coeffs() is None:
                estimator_coeff_vectors = [torch.zeros_like(coeff) for coeff in model.constraint_coeffs(sparse=True, scaled=False)] # It doesnt exist before we start sparsity, so we use zeros
            else:
                estimator_coeff_vectors = model.estimator_coeffs()

            board.write(iteration, loss, MSE, Reg, l1_norm, model.constraint_coeffs(sparse=True, scaled=True), model.constraint_coeffs(sparse=True, scaled=False), estimator_coeff_vectors)
            
        # ================== Validation and sparsity =============
        # Updating sparsity and or convergence
        sparsity_scheduler(iteration, torch.sum(l1_norm))
        if sparsity_scheduler.apply_sparsity is True:
            with torch.no_grad():
                model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)
                sparsity_scheduler.reset()
                print(model.sparsity_masks)

        # Checking convergence
        convergence(iteration, torch.sum(l1_norm))
        if convergence.converged is True:
            print('Algorithm converged. Stopping training.')
            break
    board.close()

In [19]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

In [20]:
sparsity_scheduler = s_periodic(patience=100, delta=0.05, periodicity=100)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), amsgrad=True) # Defining optimizer
    
train_static(model, X, y, 300, optimizer, sparsity_scheduler, write_iterations=25, log_dir='runs/multitask_static/', max_iterations=10000, delta=0.01, patience=100) # Running

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       2550     25.50%             412s   1.27e-02   1.25e-02   3.64e-07   3.91e+00 [tensor([False,  True,  True,  True, False, False,  True, False,  True, False,
        False, False])]
       2650     26.50%             405s   1.88e-02   1.81e-02   2.26e-06   1.36e+00 [tensor([False,  True, False, False, False, False, False, False, False, False,
        False, False])]
       2750     27.50%             397s   1.80e-02   1.73e-02   2.65e-06   1.00e+00 [tensor([False,  True, False, False, False, False, False, False, False, False,
        False, False])]
Algorithm converged. Stopping training.
